In [1]:
#Same accross all Model Selection notebooks
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline

#Hyperparameter search method, metric: "roc_auc"
from sklearn.model_selection import GridSearchCV
#metric
from sklearn.metrics import roc_auc_score as auc

#Different for different Model Selection notebooks
#preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
#model
from sklearn.neighbors import KNeighborsClassifier

In [85]:
#vvvvvvvvvvvvvv
model_name="KNeighborsClassifier"
#^^^^^^^^^^^^^^
#select the index of file to load


train_files=["DL_train.csv", #0
             "Morgan_train.csv", #1
             "Both_train.csv"] #2

test_files=["DL_test.csv" #0
            ,"Morgan_test.csv" #1
            ,"Both_test.csv"] #2

dataset_names=["DL", #0
               "Morgan", #1
               "Both"] #2

#vvvvvvvvvvvvvv
data_index=1
#^^^^^^^^^^^^^^

norm_type = [StandardScaler(), #1
             MinMaxScaler()]  #2


#vvvvvvvvvvvvvv
norm_index=0
#^^^^^^^^^^^^^^

models=[
    KNeighborsClassifier,#0
    
]

model_calls=[m() for m in models]


#vvvvvvvvvvvvvv
model_index=0
#^^^^^^^^^^^^^^

#vvvvvvvvvvvvvv
param_grid = {
    "model__n_neighbors":[1,2]
}
#^^^^^^^^^^^^^^



# 1. Import the train and test datasets *

In [86]:


dataset_name=dataset_names[data_index]
#check dataset if there is a index col, id there is include the argument index_col=0
x_train = pd.read_csv(train_files[data_index],index_col=0)
y_train = x_train.ACTIVE
x_train.drop(["ACTIVE"],axis=1,inplace=True)
x_test = pd.read_csv(test_files[data_index],index_col=0) 

# 2. Setup Pipeline *

In [87]:
#Step 1: Normalise the data

#Step 2: Model fit


#Step 3: Pipeline
pipeline=Pipeline(steps=[("norm",norm_type[norm_index]),("model",model_calls[model_index])])

# 3. Setup Parameter Grid *

In [88]:
param_grid

{'model__n_neighbors': [1, 2]}

# 4. Setup GridSearch

In [89]:
score_metric = "roc_auc"
search = GridSearchCV(pipeline,param_grid,
                      n_jobs=-1,
                      cv=10,
                      scoring=score_metric)


In [90]:
search.fit(x_train,y_train)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('norm', StandardScaler()),
                                       ('model', KNeighborsClassifier())]),
             n_jobs=-1, param_grid={'model__n_neighbors': [1, 2]},
             scoring='roc_auc')

In [91]:
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

Best parameter (CV score=0.613):
{'model__n_neighbors': 2}


In [92]:
search_res = pd.DataFrame(search.cv_results_)
search_res = search_res.sort_values(by=['rank_test_score'])
search_res.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
1,0.998925,0.091693,34.232973,9.043257,2,{'model__n_neighbors': 2},0.620648,0.603560,0.617228,0.603787,0.591474,0.606271,0.645974,0.602952,0.620257,0.620391,0.613254,0.014331,1
0,1.646535,0.244593,39.680984,1.092205,1,{'model__n_neighbors': 1},0.591978,0.583405,0.586101,0.586522,0.571618,0.589088,0.622961,0.571035,0.591752,0.591946,0.588641,0.013594,2


# 5.Estimation of Model Performance based on CV in GridSearch

In [93]:
estimated_auc = float(search_res.query("rank_test_score == 1").mean_test_score)

# 6.Best Hyperparameter *

In [94]:
parms = search_res.query("rank_test_score == 1").params

In [95]:
parms

1    {'model__n_neighbors': 2}
Name: params, dtype: object

In [96]:
parms.values

array([{'model__n_neighbors': 2}], dtype=object)

In [98]:
n_neighbors = parms.values[0]["model__n_neighbors"]
n_neighbors

2

# 7.Train model on full train dataset

In [ ]:
full_model = models[model_index](n_neighbors=n_neighbors)

In [ ]:
full_model.fit(x_train,y_train)

# 8.Prediction score for test dataset

In [ ]:
pred_test = pd.DataFrame(full_model.predict_proba(x_test))[1]


In [ ]:
pred_test.head()

# 9.Export result

In [ ]:
result = pd.DataFrame([estimated_auc,])

In [ ]:
result.head()

In [ ]:
result = pd.concat([result,pred_test])

In [ ]:
result.head()

In [48]:
result.to_csv(f"{model_name}_{dataset_name}_Result.csv")